# DIOR

In [1]:
# -------------------
# 1) DIOR Sephora (Bazaarvoice)
# -------------------
import requests
import pandas as pd
from urllib.parse import urljoin

sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP440962&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
sephora_json = requests.get(sephora_url, timeout=60).json()
sephora_reviews = sephora_json.get("Results", [])
products_info = (sephora_json.get("Includes") or {}).get("Products", {})


dior_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": 'Dior Forever Skin Glow',
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "77 CAD",   # <-- added
} for r in sephora_reviews])


# -------------------
# 2) DIOR Shoppers (PowerReviews)
# -------------------
shoppers_url = (
    "https://display.powerreviews.com/m/867678/l/all/product/3348901611282/reviews"
    "?apikey=f40b12ae-7efa-423a-8f12-cb902debf1c0&_noconfig=true&page_locale=en_CA"
)
shoppers_json = requests.get(shoppers_url, timeout=60).json()
results = shoppers_json.get("results", [])
block = results[0] if results else {}
shoppers_reviews = block.get("reviews", [])

df_shoppers_raw = pd.json_normalize(shoppers_reviews, sep=".")

shoppers_time = pd.to_datetime(
    pd.to_numeric(df_shoppers_raw.get("details.created_date"), errors="coerce"),
    unit="ms", errors="coerce", utc=True
)

rating_series = pd.Series([None] * len(df_shoppers_raw))
if "metrics.rating" in df_shoppers_raw:
    rating_series = df_shoppers_raw["metrics.rating"]
elif "details.rating" in df_shoppers_raw:
    rating_series = df_shoppers_raw["details.rating"]


dior_shoppers = pd.DataFrame({
    "source": "powerreviews",
    "retailer": "shoppers_ca",
    "product_id": block.get("page_id"),
    "product_name": 'Dior Forever Skin Glow',
    "rating": pd.to_numeric(rating_series, errors="coerce"),
    "submission_time": shoppers_time,
    "user_nickname": df_shoppers_raw.get("details.nickname"),
    "review_text": df_shoppers_raw.get("details.comments"),
    "price": "77 CAD",   # <-- added
})

# -------------------
# 3) DIOR Ulta (PowerReviews, multiple pages)
# -------------------
base = "https://display.powerreviews.com"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"
url = f"{base}/m/6406/l/en_US/product/pimprod2036824/reviews?apikey={api_key}&_noconfig=true&page_locale=en_US"

ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}
max_pages = 20

page_count = 0
while True:
    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break

    df_ulta_raw = pd.json_normalize(reviews, sep=".")
    n_ulta = len(df_ulta_raw)

    ul_num = pd.to_numeric(df_ulta_raw.get("details.created_date", pd.Series([None]*n_ulta)), errors="coerce")
    ul_ms = pd.to_datetime(ul_num.where(ul_num >= 1e12), unit="ms", errors="coerce", utc=True)
    ul_s  = pd.to_datetime(ul_num.where(ul_num <  1e12), unit="s",  errors="coerce", utc=True)
    ul_time = ul_ms.combine_first(ul_s)

    ul_disp = df_ulta_raw.get("details.nickname", pd.Series([None]*n_ulta))
    if "reviewer.display_name" in df_ulta_raw:
        ul_disp = df_ulta_raw["reviewer.display_name"].combine_first(ul_disp)
    if "reviewer.name" in df_ulta_raw:
        ul_disp = ul_disp.combine_first(df_ulta_raw["reviewer.name"])

    ul_text = df_ulta_raw.get("details.comments", pd.Series([None]*n_ulta)).combine_first(
        df_ulta_raw.get("review_text", pd.Series([None]*n_ulta))
    )

    if "metrics.rating" in df_ulta_raw:
        ul_rating = df_ulta_raw["metrics.rating"]
    elif "details.rating" in df_ulta_raw:
        ul_rating = df_ulta_raw["details.rating"]
    else:
        ul_rating = pd.Series([None]*n_ulta)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": "pimprod2036824",
        "product_name": "Dior Forever Skin Glow",
        "rating": pd.to_numeric(ul_rating, errors="coerce"),
        "submission_time": ul_time,
        "user_nickname": ul_disp,
        "review_text": ul_text,
        "price": "60 USD",   # <-- added here
    })
    ulta_rows.append(df_tmp)

    page_count += 1
    if page_count >= max_pages:
        break

    paging = js.get("paging") or block.get("paging") or {}
    next_rel = paging.get("next_page_url")
    if not next_rel:
        break
    url = urljoin(base, next_rel)
    if "apikey=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}apikey={api_key}"
    if "_noconfig=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}_noconfig=true"

dior_ulta = pd.concat(ulta_rows, ignore_index=True)


# -------------------
# COMBINE ALL (Dior)
# -------------------
dior_all = pd.concat([dior_sephora, dior_shoppers, dior_ulta], ignore_index=True)


print(dior_all["retailer"].value_counts())


dior_all.head()

retailer
sephora_ca     100
ulta_us        100
shoppers_ca      3
Name: count, dtype: int64


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2558047,Dior Forever Skin Glow,4,2025-11-27 20:10:02+00:00,JulieBBeans,Nice hydrating foundation. A thin layer goes a...,77 CAD
1,bazaarvoice,sephora_ca,2558005,Dior Forever Skin Glow,5,2025-11-04 05:56:27+00:00,None,"As someone who does not wear makeup often, thi...",77 CAD
2,bazaarvoice,sephora_ca,2558112,Dior Forever Skin Glow,5,2025-10-31 16:33:35+00:00,PatriceAhl,I love this foundation so much!! I struggle to...,77 CAD
3,bazaarvoice,sephora_ca,2558112,Dior Forever Skin Glow,3,2025-10-30 15:30:06+00:00,manpreet1,Really good Dior hydration foundation but it’s...,77 CAD
4,bazaarvoice,sephora_ca,2558112,Dior Forever Skin Glow,5,2025-10-29 12:52:07+00:00,jenn2080,Beat foundation ever pricey but worth it. Will...,77 CAD


# NARS

In [2]:
import requests
import pandas as pd

base = "https://display.powerreviews.com/m/6406/l/en_US/product/pimprod2030141/reviews"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"

nars_ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop over offsets (0, 5, 10, ... 100)
for offset in range(0, 96, 5):
    url = (
        f"{base}?paging.from={offset}&paging.size=5"
        f"&filters=&search=&sort=Newest&image_only=false"
        f"&page_locale=en_US&native_only=true&_noconfig=true&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break  # stop if no more reviews

    df_raw = pd.json_normalize(reviews, sep=".")

    # handle time (ms or s)
    ul_num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ul_ms = pd.to_datetime(ul_num.where(ul_num >= 1e12), unit="ms", errors="coerce", utc=True)
    ul_s  = pd.to_datetime(ul_num.where(ul_num <  1e12), unit="s",  errors="coerce", utc=True)
    ul_time = ul_ms.combine_first(ul_s)

    # reviewer name
    ul_disp = df_raw.get("details.nickname", pd.Series([None]*len(df_raw)))
    if "reviewer.display_name" in df_raw:
        ul_disp = df_raw["reviewer.display_name"].combine_first(ul_disp)
    if "reviewer.name" in df_raw:
        ul_disp = ul_disp.combine_first(df_raw["reviewer.name"])

    # review text
    ul_text = df_raw.get("details.comments", pd.Series([None]*len(df_raw))).combine_first(
        df_raw.get("review_text", pd.Series([None]*len(df_raw)))
    )

    # rating
    if "metrics.rating" in df_raw:
        ul_rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        ul_rating = df_raw["details.rating"]
    else:
        ul_rating = pd.Series([None]*len(df_raw))

    nars_df = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": "pimprod2030141",
        "product_name": "NARS Light Reflecting Foundation",
        "rating": pd.to_numeric(ul_rating, errors="coerce"),
        "submission_time": ul_time,
        "user_nickname": ul_disp,
        "review_text": ul_text,
    })

    nars_ulta_rows.append(nars_df)

# Final DataFrame for NARS Ulta reviews
nars_ulta = pd.concat(nars_ulta_rows, ignore_index=True)

print("Reviews fetched from Ulta (NARS):", len(nars_ulta))
nars_ulta.head()


Reviews fetched from Ulta (NARS): 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text
0,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-18 14:32:42.912000+00:00,Kait,"I have a combination of oily dry skin, and I a..."
1,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-14 21:00:43.535000+00:00,Bri,I have tried other foundations and ALWAYS come...
2,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,3,2025-11-12 19:16:56.299000+00:00,Raeanna,The sales associate in your store picked out t...
3,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-07 02:34:39.689000+00:00,Oralia,I love a glow foundation however I have oily s...
4,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,4,2025-10-31 04:06:11.141000+00:00,Alo,It blends really well and its buildable


In [3]:
# Example: Add a new column named 'new_column' with some sample values
nars_ulta['price'] = '54 USD'

# Display the updated DataFrame
display(nars_ulta.head())

,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-18 14:32:42.912000+00:00,Kait,"I have a combination of oily dry skin, and I a...",54 USD
1,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-14 21:00:43.535000+00:00,Bri,I have tried other foundations and ALWAYS come...,54 USD
2,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,3,2025-11-12 19:16:56.299000+00:00,Raeanna,The sales associate in your store picked out t...,54 USD
3,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,5,2025-11-07 02:34:39.689000+00:00,Oralia,I love a glow foundation however I have oily s...,54 USD
4,powerreviews,ulta_us,pimprod2030141,NARS Light Reflecting Foundation,4,2025-10-31 04:06:11.141000+00:00,Alo,It blends really well and its buildable,54 USD


In [4]:
nars_ulta['retailer'].value_counts()

,count
retailer,
ulta_us,100


In [5]:
# -------------------
# NARS Sephora (Bazaarvoice)
# -------------------
nars_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP479338&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
nars_sephora_json = requests.get(nars_sephora_url, timeout=60).json()
nars_sephora_reviews = nars_sephora_json.get("Results", [])
products_info = (nars_sephora_json.get("Includes") or {}).get("Products", {})

nars_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "NARS Light Reflecting Foundation",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "73 CAD",   # <-- corrected price
} for r in nars_sephora_reviews])

print("NARS Sephora — number of reviews:", len(nars_sephora))
nars_sephora.head()


NARS Sephora — number of reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2514685,NARS Light Reflecting Foundation,4,2025-11-27 17:34:42+00:00,Gwen098,This has been my first experience with a highe...,73 CAD
1,bazaarvoice,sephora_ca,2514545,NARS Light Reflecting Foundation,5,2025-11-25 20:33:04+00:00,leen360,I like this foundation my skin look natural an...,73 CAD
2,bazaarvoice,sephora_ca,2514560,NARS Light Reflecting Foundation,1,2025-11-25 17:41:23+00:00,arshdeepkaur123,It doesn’t match my skin and not full coverage...,73 CAD
3,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,5,2025-11-24 01:13:57+00:00,HappyCake84,I will shout it from the rooftops that the NAR...,73 CAD
4,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,5,2025-11-23 03:36:08+00:00,k4xo,"10/10 my holy grail ! She sits so beautiful, b...",73 CAD


In [6]:
# -------------------
# NARS Shoppers (PowerReviews, multiple pages)
# -------------------
import requests
import pandas as pd

base = "https://display.powerreviews.com/m/867678/l/all/product/194251150192/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

nars_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop pages in steps of 10 reviews (paging.size=10)
for offset in range(0, 96, 10):
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # Convert timestamps (ms or sec)
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # Reviewer name
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # Review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # Rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "NARS Light Reflecting Foundation",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "73 CAD",   # <-- correct price
    })
    nars_shoppers_rows.append(df_tmp)

nars_shoppers = pd.concat(nars_shoppers_rows, ignore_index=True)

print("NARS Shoppers — number of reviews:", len(nars_shoppers))
nars_shoppers.head()


NARS Shoppers — number of reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-11-20 21:11:47.929000+00:00,James,This is my favorite foundation. It feels like ...,73 CAD
1,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-11-20 05:39:22.979000+00:00,Mayra,Ordered it for the first time. I love it 😀,73 CAD
2,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,4,2025-11-20 00:07:19.740000+00:00,Manta,It's little dry on my skin,73 CAD
3,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-11-19 23:01:50.269000+00:00,linda,my all time favorite,73 CAD
4,powerreviews,shoppers_ca,194251150192,NARS Light Reflecting Foundation,5,2025-11-19 22:41:32.842000+00:00,Shaz,There is none better foundation to give a soft...,73 CAD


In [7]:
# -------------------
# COMBINE ALL (NARS)
# -------------------
nars_all = pd.concat([nars_sephora, nars_shoppers, nars_ulta], ignore_index=True)

print("Rows saved (NARS):", len(nars_all))
print(nars_all.groupby(["retailer", "price"]).size().reset_index(name="num_reviews"))

nars_all.head()


Rows saved (NARS): 300
      retailer   price  num_reviews
0   sephora_ca  73 CAD          100
1  shoppers_ca  73 CAD          100
2      ulta_us  54 USD          100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2514685,NARS Light Reflecting Foundation,4,2025-11-27 17:34:42+00:00,Gwen098,This has been my first experience with a highe...,73 CAD
1,bazaarvoice,sephora_ca,2514545,NARS Light Reflecting Foundation,5,2025-11-25 20:33:04+00:00,leen360,I like this foundation my skin look natural an...,73 CAD
2,bazaarvoice,sephora_ca,2514560,NARS Light Reflecting Foundation,1,2025-11-25 17:41:23+00:00,arshdeepkaur123,It doesn’t match my skin and not full coverage...,73 CAD
3,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,5,2025-11-24 01:13:57+00:00,HappyCake84,I will shout it from the rooftops that the NAR...,73 CAD
4,bazaarvoice,sephora_ca,2514727,NARS Light Reflecting Foundation,5,2025-11-23 03:36:08+00:00,k4xo,"10/10 my holy grail ! She sits so beautiful, b...",73 CAD


In [8]:
nars_all['retailer'].value_counts()

,count
retailer,
sephora_ca,100
shoppers_ca,100
ulta_us,100


In [9]:
dior_all['retailer'].value_counts()

,count
retailer,
sephora_ca,100
ulta_us,100
shoppers_ca,3


# ESTEE LAUDER

In [10]:
import requests
import pandas as pd

# -------------------
# Estée Lauder Double Wear – Shoppers (PowerReviews)
# -------------------
base = "https://display.powerreviews.com/m/867678/l/all/product/027131973195/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

estee_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop through review pages
for offset in range(0, 96, 10):  # paging.size=10, step by 10
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # Time conversion
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # Reviewer
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # Review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # Rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "Estée Lauder Double Wear",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "67 CAD",   # 🔹 update to the real price at Shoppers
    })
    estee_shoppers_rows.append(df_tmp)

# Final shoppers DataFrame
estee_shoppers = pd.concat(estee_shoppers_rows, ignore_index=True)

print("Estée Lauder Double Wear — Shoppers reviews:", len(estee_shoppers))
estee_shoppers.head(6)



Estée Lauder Double Wear — Shoppers reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-11-18 20:17:17.920000+00:00,Wendy G,I have worn this foundation for years! It las...,67 CAD
1,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-11-15 03:04:37.124000+00:00,Mona,Product stays-in-place and offers very good co...,67 CAD
2,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,3,2025-11-15 02:25:27.123000+00:00,Jane,"This is my first bottle of the Double Wear, af...",67 CAD
3,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-11-11 04:05:56.404000+00:00,Hermy,I love this product at 54. Tried other brands ...,67 CAD
4,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-11-08 16:15:17.768000+00:00,D,I have been using this foundation for more tha...,67 CAD
5,powerreviews,shoppers_ca,027131973195,Estée Lauder Double Wear,5,2025-10-30 16:04:15.104000+00:00,NARINE VARDANYAN,My favorite product - double wear foundation f...,67 CAD


In [11]:
import requests
import pandas as pd
from urllib.parse import urljoin

# -------------------
# Estée Lauder Double Wear – Ulta (PowerReviews, multiple pages)
# -------------------
base = "https://display.powerreviews.com"
api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"
# product path from your URL
path = "/m/6406/l/en_US/product/xlsImpprod14641507/reviews"
url = f"{base}{path}?apikey={api_key}&_noconfig=true"

ulta_rows = []
headers = {"User-Agent": "Mozilla/5.0"}
max_pages = 20

page_count = 0
while True:
    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results")
    block = results if isinstance(results, dict) else (results[0] if results else {})
    reviews = block.get("reviews") or []
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # convert created_date
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # user nickname
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "ulta_us",
        "product_id": block.get("page_id", "xlsImpprod14641507"),
        "product_name": "Estée Lauder Double Wear",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "52 USD",   # set your Ulta price
    })
    ulta_rows.append(df_tmp)

    page_count += 1
    if page_count >= max_pages:
        break

    paging = js.get("paging") or block.get("paging") or {}
    next_rel = paging.get("next_page_url")
    if not next_rel:
        break
    url = urljoin(base, next_rel)
    if "apikey=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}apikey={api_key}"
    if "_noconfig=" not in url:
        sep = "&" if "?" in url else "?"
        url = f"{url}{sep}_noconfig=true"

estee_ulta = pd.concat(ulta_rows, ignore_index=True)

print("Estée Lauder Double Wear — Ulta reviews:", len(estee_ulta))
estee_ulta.head()

Estée Lauder Double Wear — Ulta reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-11-19 05:01:21.399000+00:00,Danielle,Great coverage with a natural look and feel.,52 USD
1,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-11-18 16:35:19.980000+00:00,Silvia,"I like this foundation, it's not cakey and it ...",52 USD
2,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-11-18 02:46:08.882000+00:00,:),I cannot say enough how much i love this found...,52 USD
3,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-11-16 21:24:11.253000+00:00,Megan,Great foundation for 40+ baddies! My go to mak...,52 USD
4,powerreviews,ulta_us,xlsImpprod14641507,Estée Lauder Double Wear,5,2025-11-16 12:59:38.791000+00:00,Mary,Best make up for a great coverage,52 USD


In [12]:
estee_ulta['retailer'].value_counts()

,count
retailer,
ulta_us,100


In [13]:
# -------------------
# Estée Lauder Double Wear – Sephora (Bazaarvoice)
# -------------------
estee_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP378284&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
estee_sephora_json = requests.get(estee_sephora_url, timeout=60).json()
estee_sephora_reviews = estee_sephora_json.get("Results", [])
products_info = (estee_sephora_json.get("Includes") or {}).get("Products", {})

estee_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "Estée Lauder Double Wear",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "70.5 CAD",  # <— update to correct Sephora price
} for r in estee_sephora_reviews])

print("Estée Lauder Double Wear — Sephora reviews:", len(estee_sephora))
estee_sephora.head()


Estée Lauder Double Wear — Sephora reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2112175,Estée Lauder Double Wear,5,2025-11-27 04:58:41+00:00,rosiewonderful,GOAT. A little goes a long way. It looks just ...,70.5 CAD
1,bazaarvoice,sephora_ca,1854454,Estée Lauder Double Wear,5,2025-11-26 17:37:05+00:00,Craly,I’ve been using this product for over 20 years...,70.5 CAD
2,bazaarvoice,sephora_ca,1854447,Estée Lauder Double Wear,5,2025-11-26 02:06:03+00:00,taniasa,Really nice foundation long wear does not smud...,70.5 CAD
3,bazaarvoice,sephora_ca,1465301,Estée Lauder Double Wear,5,2025-11-25 16:29:29+00:00,Addienac,"10/10 foundation, I get one shade for summerti...",70.5 CAD
4,bazaarvoice,sephora_ca,1465152,Estée Lauder Double Wear,5,2025-11-21 23:00:03+00:00,Frenchie333,"I have been on a foundation journey, desperate...",70.5 CAD


In [14]:
# -------------------
# COMBINE ALL (Estée Lauder Double Wear)
# -------------------
estee_all = pd.concat(
    [estee_sephora, estee_shoppers, estee_ulta],
    ignore_index=True
)

estee_all.head()

,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,2112175,Estée Lauder Double Wear,5,2025-11-27 04:58:41+00:00,rosiewonderful,GOAT. A little goes a long way. It looks just ...,70.5 CAD
1,bazaarvoice,sephora_ca,1854454,Estée Lauder Double Wear,5,2025-11-26 17:37:05+00:00,Craly,I’ve been using this product for over 20 years...,70.5 CAD
2,bazaarvoice,sephora_ca,1854447,Estée Lauder Double Wear,5,2025-11-26 02:06:03+00:00,taniasa,Really nice foundation long wear does not smud...,70.5 CAD
3,bazaarvoice,sephora_ca,1465301,Estée Lauder Double Wear,5,2025-11-25 16:29:29+00:00,Addienac,"10/10 foundation, I get one shade for summerti...",70.5 CAD
4,bazaarvoice,sephora_ca,1465152,Estée Lauder Double Wear,5,2025-11-21 23:00:03+00:00,Frenchie333,"I have been on a foundation journey, desperate...",70.5 CAD


# ARMANI

In [15]:
import requests
import pandas as pd

# -------------------
# Armani Luminous Foundation — Shoppers (PowerReviews)
# -------------------
base = "https://display.powerreviews.com/m/867678/l/all/product/3605521752710/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

armani_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}

# Loop in steps of 10 (paging.size=10)
for offset in range(0, 96, 10):
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else (results if isinstance(results, dict) else {})
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    # parse times
    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    times = ms.combine_first(s)

    # reviewer nickname / fallback
    nick = df_raw.get("details.nickname", pd.Series([None]*n))
    if "reviewer.display_name" in df_raw:
        nick = df_raw["reviewer.display_name"].combine_first(nick)
    if "reviewer.name" in df_raw:
        nick = nick.combine_first(df_raw["reviewer.name"])

    # review text
    text = df_raw.get("details.comments", pd.Series([None]*n)).combine_first(
        df_raw.get("review_text", pd.Series([None]*n))
    )

    # rating
    if "metrics.rating" in df_raw:
        rating = df_raw["metrics.rating"]
    elif "details.rating" in df_raw:
        rating = df_raw["details.rating"]
    else:
        rating = pd.Series([None]*n)

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "Armani Luminous Foundation",
        "rating": pd.to_numeric(rating, errors="coerce"),
        "submission_time": times,
        "user_nickname": nick,
        "review_text": text,
        "price": "85 CAD",  # <-- fill in the correct Shoppers price in CAD
    })

    armani_shoppers_rows.append(df_tmp)

armani_shoppers = pd.concat(armani_shoppers_rows, ignore_index=True)
armani_shoppers.head()


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-07 13:23:34+00:00,Erincraw,this is my favorite glow foundation and I love...,85 CAD
1,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-06 00:38:33+00:00,lizsingle,I have been using luminous silk natural founda...,85 CAD
2,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-06-03 18:23:24+00:00,gail,the best! i have been using it for years. it i...,85 CAD
3,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-05-28 19:18:39+00:00,Lorelei,Simply GORGEOUS. An indulgent sensation that c...,85 CAD
4,powerreviews,shoppers_ca,3605521752710,Armani Luminous Foundation,5,2024-05-25 20:55:53+00:00,Nellie,I used Crema Nuda for years and am sorry to se...,85 CAD


In [16]:
import requests
import pandas as pd

# -------------------
# Armani Luminous Foundation — Sephora (Bazaarvoice)
# -------------------
armani_sephora_url = (
    "https://api.bazaarvoice.com/data/reviews.json?"
    "Filter=ProductId%3AP393401&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
    "&Include=Products%2CComments&Stats=Reviews"
    "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
    "&apiversion=5.4&Locale=en_CA"
)
armani_sephora_json = requests.get(armani_sephora_url, timeout=60).json()
armani_sephora_reviews = armani_sephora_json.get("Results", [])
products_info = (armani_sephora_json.get("Includes") or {}).get("Products", {})

armani_sephora = pd.DataFrame([{
    "source": "bazaarvoice",
    "retailer": "sephora_ca",
    "product_id": r.get("ProductId"),
    "product_name": "Armani Luminous Foundation",
    "rating": r.get("Rating"),
    "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
    "user_nickname": r.get("UserNickname"),
    "review_text": r.get("ReviewText"),
    "price": "85 CAD",  # <-- fill in the actual price here
} for r in armani_sephora_reviews])

print("Armani Luminous — Sephora reviews:", len(armani_sephora))
armani_sephora.head()


Armani Luminous — Sephora reviews: 100


,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,bazaarvoice,sephora_ca,1359587,Armani Luminous Foundation,5,2025-11-26 22:32:29+00:00,Feera93,I love this foundation 😍ıt is look so fresh.ıt...,85 CAD
1,bazaarvoice,sephora_ca,1803170,Armani Luminous Foundation,4,2025-11-26 06:37:57+00:00,neonviolet,"The perfect foundation for my dry skin, doesn'...",85 CAD
2,bazaarvoice,sephora_ca,2079200,Armani Luminous Foundation,5,2025-11-24 02:06:21+00:00,IcyPrincess1314,"I’ve long struggled with dry skin, and no matt...",85 CAD
3,bazaarvoice,sephora_ca,1491372,Armani Luminous Foundation,4,2025-11-21 23:27:02+00:00,Lisaaaaaaaaa,A few years ago I would have loved this but I ...,85 CAD
4,bazaarvoice,sephora_ca,1803188,Armani Luminous Foundation,5,2025-11-21 17:43:40+00:00,Neen88,I have dabbled with wrong shades of this found...,85 CAD


# FENTY , LANCOME

In [29]:
import requests
import pandas as pd
from urllib.parse import urljoin


# ----------------------------------------------------
# 1) Bazaarvoice (Sephora)
# ----------------------------------------------------
def scrape_bazaarvoice(product_id, product_name, price, locale="en_CA"):
    url = (
        "https://api.bazaarvoice.com/data/reviews.json?"
        f"Filter=ProductId%3A{product_id}&Sort=SubmissionTime%3Adesc&Limit=100&Offset=0"
        "&Include=Products%2CComments&Stats=Reviews"
        "&passkey=calXm2DyQVjcCy9agq85vmTJv5ELuuBCF2sdg4BnJzJus"
        "&apiversion=5.4"
        f"&Locale={locale}"
    )

    js = requests.get(url, timeout=60).json()
    reviews = js.get("Results", [])

    rows = []
    for r in reviews:
        rows.append({
            "source": "bazaarvoice",
            "retailer": "sephora_ca",
            "product_id": r.get("ProductId"),
            "product_name": product_name,
            "rating": r.get("Rating"),
            "submission_time": pd.to_datetime(r.get("SubmissionTime"), errors="coerce", utc=True),
            "user_nickname": r.get("UserNickname"),
            "review_text": r.get("ReviewText"),
            "price": price,
        })
    return pd.DataFrame(rows)


# ----------------------------------------------------
# 2) PowerReviews — *single page* (Shoppers)
# ----------------------------------------------------
def scrape_powerreviews_shoppers(url, product_name, price):
    js = requests.get(url, timeout=60).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) and results else results
    reviews = block.get("reviews", [])
    if not reviews:
        return pd.DataFrame()

    df = pd.json_normalize(reviews, sep=".")

    num = pd.to_numeric(df.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    time = ms.combine_first(s)

    rating = pd.to_numeric(df.get("metrics.rating", df.get("details.rating")), errors="coerce")

    return pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": product_name,
        "rating": rating,
        "submission_time": time,
        "user_nickname": df.get("details.nickname"),
        "review_text": df.get("details.comments"),
        "price": price,
    })


# ----------------------------------------------------
# 3) PowerReviews — Ulta *multipage*
# ----------------------------------------------------
def scrape_powerreviews_ulta(base_url, product_name, price, max_pages=20):
    api_key = "daa0f241-c242-4483-afb7-4449942d1a2b"
    base = "https://display.powerreviews.com"

    if "apikey=" not in base_url:
        url = f"{base_url}&apikey={api_key}&_noconfig=true"
    else:
        url = base_url

    headers = {"User-Agent": "Mozilla/5.0"}
    rows = []
    page_no = 0

    while True:
        js = requests.get(url, timeout=60, headers=headers).json()
        results = js.get("results", [])
        block = results[0] if isinstance(results, list) and results else results
        reviews = block.get("reviews", [])
        if not reviews:
            break

        df = pd.json_normalize(reviews, sep=".")
        n = len(df)

        num = pd.to_numeric(df.get("details.created_date"), errors="coerce")
        ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
        s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
        time = ms.combine_first(s)

        rating = pd.to_numeric(df.get("metrics.rating", df.get("details.rating")), errors="coerce")
        nick = df.get("details.nickname")
        text = df.get("details.comments")

        rows.append(pd.DataFrame({
            "source": "powerreviews",
            "retailer": "ulta_us",
            "product_id": block.get("page_id"),
            "product_name": product_name,
            "rating": rating,
            "submission_time": time,
            "user_nickname": nick,
            "review_text": text,
            "price": price,
        }))

        page_no += 1
        if page_no >= max_pages:
            break

        paging = js.get("paging") or block.get("paging") or {}
        next_rel = paging.get("next_page_url")
        if not next_rel:
            break

        url = urljoin(base, next_rel)
        if "apikey=" not in url:
            url += f"&apikey={api_key}"
        if "_noconfig=" not in url:
            url += "&_noconfig=true"

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame()


# ----------------------------------------------------
# SCRAPE PRODUCTS
# ----------------------------------------------------

# ---------- FENTY ----------
fenty_sephora = scrape_bazaarvoice(
    product_id="P87985432",
    product_name="Fenty Pro Filt'r Soft Matte Foundation",
    price="54 CAD"
)

fenty_ulta = scrape_powerreviews_ulta(
    base_url="https://display.powerreviews.com/m/6406/l/en_US/product/pimprod2029862/reviews?_noconfig=true",
    product_name="Fenty Pro Filt'r Soft Matte Foundation",
    price="40 USD"
)

fenty_all = pd.concat([fenty_sephora, fenty_ulta], ignore_index=True)

# ---------- LANCÔME ----------
lancome_sephora = scrape_bazaarvoice(
    product_id="P308201",
    product_name="Lancôme Teint Idole Ultra Wear Foundation",
    price="76 CAD"
)

lancome_ulta = scrape_powerreviews_ulta(
    base_url="https://display.powerreviews.com/m/6406/l/en_US/product/pimprod2039028/reviews?_noconfig=true",
    product_name="Lancôme Teint Idole Ultra Wear Foundation",
    price="57 USD"
)


lancome_shoppers_rows = []
headers = {"User-Agent": "Mozilla/5.0"}
base = "https://display.powerreviews.com/m/867678/l/all/product/3614273793049/reviews"
api_key = "f40b12ae-7efa-423a-8f12-cb902debf1c0"

for offset in range(0, 100, 10):  # 0,10,20,...90  (10 pages × 10 reviews each)
    url = (
        f"{base}?paging.from={offset}&paging.size=10"
        "&filters=&search=&sort=Newest&image_only=false"
        "&page_locale=en_CA&_noconfig=true"
        f"&apikey={api_key}"
    )

    js = requests.get(url, timeout=60, headers=headers).json()
    results = js.get("results", [])
    block = results[0] if isinstance(results, list) else results
    reviews = block.get("reviews", [])
    if not reviews:
        break

    df_raw = pd.json_normalize(reviews, sep=".")
    n = len(df_raw)

    num = pd.to_numeric(df_raw.get("details.created_date"), errors="coerce")
    ms = pd.to_datetime(num.where(num >= 1e12), unit="ms", errors="coerce", utc=True)
    s = pd.to_datetime(num.where(num < 1e12), unit="s", errors="coerce", utc=True)
    time = ms.combine_first(s)

    nick = df_raw.get("details.nickname")
    text = df_raw.get("details.comments")
    rating = pd.to_numeric(df_raw.get("metrics.rating", df_raw.get("details.rating")), errors="coerce")

    df_tmp = pd.DataFrame({
        "source": "powerreviews",
        "retailer": "shoppers_ca",
        "product_id": block.get("page_id"),
        "product_name": "Lancôme Teint Idole Ultra Wear Foundation",
        "rating": rating,
        "submission_time": time,
        "user_nickname": nick,
        "review_text": text,
        "price": "63 CAD",
    })

    lancome_shoppers_rows.append(df_tmp)

lancome_shoppers = pd.concat(lancome_shoppers_rows, ignore_index=True)

print("Lancome — Shoppers reviews:", len(lancome_shoppers))


lancome_all = pd.concat([lancome_sephora, lancome_ulta, lancome_shoppers], ignore_index=True)




Lancome — Shoppers reviews: 100


In [25]:
# -------------------
# COMBINE ALL (Armani Luminous Foundation)
# -------------------
armani_all = pd.concat(
    [armani_sephora, armani_shoppers],   # add armani_ulta here later if you have it
    ignore_index=True
)


In [30]:
# -------------------
# COMBINE ALL BRANDS
# -------------------

all_reviews = pd.concat(
    [dior_all, nars_all, estee_all, armani_all, lancome_all, fenty_all ],
    ignore_index=True
)

print("Total rows (all brands):", len(all_reviews))


Total rows (all brands): 1503


In [31]:
all_reviews['product_name'].value_counts()

,count
product_name,
NARS Light Reflecting Foundation,300
Estée Lauder Double Wear,300
Lancôme Teint Idole Ultra Wear Foundation,300
Dior Forever Skin Glow,203
Armani Luminous Foundation,200
Fenty Pro Filt'r Soft Matte Foundation,200


In [33]:
lancome_ulta.head()

,source,retailer,product_id,product_name,rating,submission_time,user_nickname,review_text,price
0,powerreviews,ulta_us,pimprod2039028,Lancôme Teint Idole Ultra Wear Foundation,5,2025-11-17 02:17:24.501000+00:00,Sheri,"Okay this foundation is expensive, but after u...",57 USD
1,powerreviews,ulta_us,pimprod2039028,Lancôme Teint Idole Ultra Wear Foundation,5,2025-11-11 21:55:28.386000+00:00,Brittany,[I received this product in exchange for my ho...,57 USD
2,powerreviews,ulta_us,pimprod2039028,Lancôme Teint Idole Ultra Wear Foundation,5,2025-11-11 17:15:46.556000+00:00,Jo,This is the only foundation I use. It covers a...,57 USD
3,powerreviews,ulta_us,pimprod2039028,Lancôme Teint Idole Ultra Wear Foundation,5,2025-11-10 19:00:19.994000+00:00,Sabrina,I had been looking for a full coverage foundat...,57 USD
4,powerreviews,ulta_us,pimprod2039028,Lancôme Teint Idole Ultra Wear Foundation,5,2025-11-09 20:41:06.038000+00:00,Arnela,I finally found the right foundation and prime...,57 USD


In [32]:
print("Lancome — Sephora count:", len(lancome_sephora))
print("Lancome — Ulta count:", len(lancome_ulta))
print("Lancome — Shoppers count:", len(lancome_shoppers))

print("\nTotal Lancome reviews:", len(lancome_all))


Lancome — Sephora count: 100
Lancome — Ulta count: 100
Lancome — Shoppers count: 100

Total Lancome reviews: 300


In [34]:
all_reviews.to_csv('all_reviews.csv', index=False)

print("✅ File saved as all_reviews.csv")

✅ File saved as all_reviews.csv
